In [1]:
import nltk
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from nltk.corpus import stopwords
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import urllib3

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alfredo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df2 = pd.read_csv('CoronaDatabase.csv')

In [10]:
df2.head()

,noticia,classificacao
0,Café previne o coronavírus,1
1,Beber água de 15 em 15 minutos cura o coronavírus,1
2,Chá de limão com bicarbonato quente cura coron...,1
3,Ministro da Saúde pede para compartilhar áudio...,1
4,"Aplicativo Coronavírus-SUS, do Governo do Bras...",1


In [11]:
df2 = df2[['noticia', 'classificacao']]
df2 = df2.rename(columns={"noticia": "noticia", "classificacao": "label"})
df2['label'] = df2['label'].replace({0: 'true', 1: 'fake'})
df2.tail()
#df = df.append(df2, ignore_index=True)

,noticia,label
184,Vitamina D não mata coronavírus,true
185,Vitamina C + zinco não cura novo coronavírus,true
186,Uísque e mel não protege contra coronavírus,true
187,Colocar luvas para manusear dinheiro e protege...,true
188,Colocar luvas para manusear dinheiro e previne...,true


In [190]:
x_train,x_test,y_train,y_test=train_test_split(df2['noticia'], df2['label'], test_size=0.2,random_state=10)

In [191]:
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True,
                                   max_df=.5,
                                   lowercase =True,
                                   stop_words=stopwords.words('portuguese'))
tfidf_train = tfidf_vectorizer.fit_transform(x_train)
tfidf_test = tfidf_vectorizer.transform(x_test)

In [192]:
selector = SelectPercentile(f_classif, percentile=100)
selector.fit(tfidf_train, y_train)

SelectPercentile(percentile=100,
                 score_func=<function f_classif at 0x00000097B9C86EE8>)

In [193]:
features_train = selector.transform(tfidf_train).toarray()
features_test = selector.transform(tfidf_test).toarray()

In [194]:
model = SVC(kernel='rbf', C=6000, gamma=.0003)
model.fit (features_train, y_train)

y_pred = model.predict(features_test)
score = accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 92.11%


In [184]:
lr = LogisticRegression(C=10)
lr.fit (features_train, y_train)

lry_pred = lr.predict(features_test)
score = accuracy_score(y_test,lry_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 89.47%


In [162]:
df3 = pd.read_csv('CoronaResult.csv')

In [163]:
df3

,noticia,classificacao
0,beber água quente não mata o coronavírus,0
1,china lança vacina contra o coronavírus,1
2,o Brasil ultrapassa um milhão de infectados pe...,0
3,o coronavirus se espalha pelo ar,1
4,vitamina c não protege contra coronavirus,0
5,Colocar luvas ao mexer em dinheiro e evitar co...,0
6,beber água quente mata o coronavírus,1
7,café previne coronavirus,1


In [164]:
tfidf_target = tfidf_vectorizer.transform(df3['noticia'])

In [165]:
features_target = selector.transform(tfidf_target).toarray()

In [166]:
result_target = model.predict(features_target)
print(result_target)

['fake' 'fake' 'true' 'fake' 'fake' 'true' 'fake' 'fake']


In [167]:
result_target = lr.predict(features_target)
print(result_target)

['fake' 'fake' 'true' 'fake' 'fake' 'true' 'fake' 'fake']


In [147]:
for i in range(len(df3['noticia'])):
    if result_target[i]=='fake' :
        df3['classificacao'][i] = False
    else:
        df3['classificacao'][i] = True
    
display(df3)

C:\Users\alfredo\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\alfredo\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,noticia,classificacao
0,beber água quente não mata o coronavírus,False
1,china lança vacina contra o coronavírus,False
2,o Brasil ultrapassa um milhão de infectados pe...,True
3,o coronavirus se espalha pelo ar,False
4,vitamina c não protege contra coronavirus,False
5,Colocar luvas ao mexer em dinheiro e evitar co...,True
6,beber água quente mata o coronavírus,False
7,café previne coronavirus,False
